# ArcGIS Online CAT Item Audit

Description: This will audit all of the items in the AGO organization.
 
Created on: 2/16/2018
 
Purpose:This script will go through an entire ArcGIS Online Organization or a Portal Organization and look through all of the Items. Then, this script will check the all of the parameters of all of the Items. Then all of the parameters will be written to a csv file. The csv file can then be used to aid the administrator in the cleanup of the Items within the Organization. This is a Jupyter Notebook written using the ArcGIS Python API.
 
Authored by: Joe Guzi
 
Previous Production Date: 2/23/24      

Production Date: 2/25/25

Notes:
- 8/12/24 --- added Audit Template
- 2/23/25 --- Updated for the ArcGIS Python API version 2.4 update
- 2/25/25 --- Added optional code to upload results to AGO
- 11/11/25 --- Added field for Last Viewed Date

# Step 1: Import Libraries

In [ ]:
# import
print("Import Libraries...")
from arcgis.gis import GIS
from arcgis.gis import ItemProperties
from arcgis.gis import ItemTypeEnum
import getpass
from IPython.display import display
import json
import requests
import csv
import time
from tkinter.filedialog import asksaveasfilename
print("Import Libraries Complete!")

# Step 2: Connect to ArcGIS Online or ArcGIS Enterprise

Use one of the methods below to conenct to ArcGIS Online or ArcGIS Enterprise.

## Connect with current AGO Credentials to this organization

Use this option if you are already signed into AGO and are running this notebook as a hosted notebook in AGO.

In [ ]:
gis = GIS("home")
gis
print("Connect to ArcGIS Online Complete!")

## Connect to ArcGIS Online by providing Org URL and Credentials (built-in users)

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
User = input("What's your  username? ")
Password = getpass.getpass("What's your password? ")

# Connection 
gis = GIS(Organization, User, Password)
gis
print("Connected to AGO")

## Connect to ArcGIS Online by providing a profile

We connect by using a profile. This profile has been setup beforehand by using built-in credentials and registering it with the profile itself. We have a section in the documentation that shows you how to do this if you are interested.

In [ ]:
# Connection Variables
gis = GIS(profile="your_online_profile") # edit "your_online_profile" with the name of your profile
gis
print("Connected to AGO")

## User authentication with OAuth 2.0

https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#user-authentication-with-oauth-20

In [ ]:
# Connection Variables
Organization = input("What's your organization? ")
OrgClientID = input("What's your  ClientID? ")
gis = GIS(Organization, client_id=OrgClientID)
print("Successfully logged in as: " + gis.properties.user.username)
print("Connected to AGO")

## Connect to ArcGIS Enterprise with Single Sign On

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
gis = GIS(Organization)
gis
print("Logged in as " + gis.properties.user.username + " to " + gis.properties.portalName)
print("Connected to AGE")

# Step 3: Define Functions (Optional)

In [ ]:
print("Begin Function Section.")
# This is where you define Functions

print("Function Section Complete!")

# Step 4: Execute Code Section

In [ ]:
print("Begin Code Section.")
Audit = gis.content.search(query="", max_items = 10000)
t = ""
for Item in Audit:
    ItemID = str(Item.get("id"))
    ItemOwner = str(Item.get("owner"))
    CreatedTime = str(Item.get("created"))
    if int(CreatedTime)<0:
        GroupModified = ""
    else:
        realtime = float(CreatedTime[:-3])
        ItemCreated = str(time.strftime("%b %d %Y %H:%M:%S", time.gmtime(realtime)))

    ModifiedTime = str(Item.get("modified"))
    if int(ModifiedTime)<0:
        ItemModified = ""
    else:
        realtime = float(ModifiedTime[:-3])
        ItemModified = str(time.strftime("%b %d %Y %H:%M:%S", time.gmtime(realtime)))
    ItemName = str(Item.get("name"))
    ItemTitle = str(Item.get("title"))#.replace(',', ""))
    ItemType = str(Item.get("type"))
    ItemTypeKeywords = str(Item.get("typeKeywords"))
    ItemDescription = str(Item.get("description"))
    ItemTags = str(Item.get("tags"))
    ItemSnippet = str(Item.get("snippet"))
    ItemThumbnail = str(Item.get("thumbnail"))
    ItemExtent = str(Item.get("extent"))
    ItemSpatialReference = str(Item.get("spatialReference"))
    ItemAccessInformation = str(Item.get("accessInformation"))
    ItemLicenseInfo = str(Item.get("licenseInfo"))
    ItemCulture = str(Item.get("culture"))
    ItemURL = str(Item.get("url"))
    ItemAccess = str(Item.get("access"))
    ItemSize = str(Item.get("size"))
    ItemListed= str(Item.get("listed"))
    ItemNumComments = str(Item.get("numComments"))
    ItemNumRatings = str(Item.get("numRatings"))
    ItemAvgRating = str(Item.get("avgRating"))
    ItemNumViews = str(Item.get("numViews"))
    LastViewedTime = str(Item.get("lastViewed"))
    if int(LastViewedTime)<0:
        ItemLastViewed = ""
    else:
        realtime = float(LastViewedTime[:-3])
        ItemLastViewed = str(time.strftime("%b %d %Y %H:%M:%S", time.gmtime(realtime)))
    ItemContentStatus = str(Item.get("contentStatus"))

    t += ItemID + ","
    t += ItemOwner + ","
    t += ItemCreated + ","
    t += ItemModified + ","
    t += ItemName.replace('℠', "") + ","
    ItemTitle = ItemTitle.replace(",", ";")
    t += ItemTitle.replace('℠', "") + ","
    t += ItemType + ","
    t += ItemTypeKeywords.replace(',', "|") + ","
    t += " " + "," #ItemDescription.replace('℠', "")  + ","
    t += ItemTags.replace(',', "|") + ","
    ItemSnippet = ItemSnippet.replace(",", ";")
    ItemSnippet = ItemSnippet.replace("\n", ";")
    t += ItemSnippet.replace('℠', "") + ","
    t += ItemThumbnail + ","
    t += ItemExtent.replace(',', ";") + ","
    t += ItemSpatialReference + ","
    ItemAccessInformation = ItemAccessInformation.replace('<p>', " ")
    ItemAccessInformation = ItemAccessInformation.replace('</p>', " ")
    ItemAccessInformation = ItemAccessInformation.replace('\n', " ")
    t += ItemAccessInformation.replace(',', ";") + ","
    ItemLicenseInfo = ItemLicenseInfo.replace('<p>', " ")
    ItemLicenseInfo = ItemLicenseInfo.replace('</p>', " ")
    ItemLicenseInfo = ItemLicenseInfo.replace('\n', " ")
    t += " " + "," #ItemLicenseInfo.replace(',', ";") + ","
    t += ItemCulture + ","
    t += ItemURL + ","
    t += ItemAccess + ","
    t += ItemSize + ","
    t += ItemListed + ","
    t += ItemNumComments + ","
    t += ItemNumRatings + ","
    t += ItemAvgRating + ","
    t += ItemNumViews + ","
    t += ItemLastViewed + ","
    t += ItemContentStatus + ","
    t += "\n"
print("Code Section Complete!")

# Step 5: Write the Results to Excel (Optional)

In [ ]:
print("Begin Write Results to Excel Section.") 
filepath = asksaveasfilename() # show an "Open" dialog box and return the path to the selected file
print(filepath)
csvfile = filepath + ".csv"
print(csvfile)
with open(csvfile, 'w') as output:
    print ("Wrtie data To CSV File")
    output.write("id,owner,created,modified,name,title,type,typeKeywords,description,tags,snippet,thumbnail,extent,spatialReference,accessInformation,licenseInfo,culture,url,access,size,listed,numComments,numRatings,avgRatings,numViews,LastViewed,ContentStatus\n")
    output.writelines(t)
    print("Wrtie data To CSV File Complete!")

# Step 6: Upload to AGO (Optional)

In [ ]:
print("Begin Upload to AGO Section.") 
file = filepath.rpartition('/')
#print(file)
filename = file[2]
#print(file[2])
testfile = gis.content.search(query="title:"+filename, item_type="Feature Layer Collection")
if testfile:
    print("Found the service")
    testfile = testfile[0]
    display(testfile)
    from arcgis.features import FeatureLayerCollection
    testfile_collection = FeatureLayerCollection.fromitem(testfile)
    print("The table already exists")
    testfile_collection.manager.overwrite(csvfile)
    display
else:
    print("We have to upload it")
    print("Define User")
    User = gis.users.me.username
    Folder = input("What folder would you like to create your item?")
    # Check if Folder Exists
    FolderExists = gis.content.folders.get(folder=Folder, owner=User)
    print(FolderExists)
    if FolderExists == None:
        print("Create Folder")
        gis.content.folders.create(Folder)
    ItemFolder = gis.content.folders.get(folder=Folder, owner=User)
    print("Define Item Properties")
    item_props = ItemProperties(title=filename + "_csv", item_type=ItemTypeEnum.CSV.value)
    print("Add csv to AGO")
    csv_future_item = ItemFolder.add(item_properties=item_props, file=csvfile)
    csv_item = csv_future_item.result()
    display(csv_item)
    print("Publish csv as Table Layer")
    csv_lyr = csv_item.publish()
    display(csv_lyr)
    
print ("The CSV has been published.\nAll done, thanks for playing!")